## Étude

In [ ]:
import numpy as np
import random
import cv2

#### NPS (Non-parametric sampling)

Algorithme:

In [16]:
import cv2
import numpy as np
import random

def growImage(sample_image, image, window_size):
    max_error = 0.3
    while not myCountNonZero(image) == image.shape[0] * image.shape[1]:
        pixel_list = getUnfilledNeighbours(image)
        if not pixel_list:
            break
        progress = 0
        for pixel in pixel_list:
            cv2.imshow("progress", image)
            cv2.waitKey(1)
            print(f"current pixel : {pixel[0]}x, {pixel[1]} y")
            window = np.array(getNeighbourWindow(image, pixel, window_size))
            ssd = normalizedSsd(sample_image, window)
            matches = findMatches2(ssd, 0.1)
            if matches:
                N = matches[0].shape[0]
                select = np.random.choice(np.arange(N))
                selected_pixel = (matches[0][select], matches[1][select])
                selected_pixel = (selected_pixel[0] + window_size // 2, selected_pixel[1] + window_size // 2)
                for ch in range(3):
                    image[pixel[0], pixel[1], ch] = sample_image[selected_pixel[0], selected_pixel[1], ch]
            cv2.destroyAllWindows()

def findMatches2(ssd, error):
    min_ssd = np.min(ssd)
    min_threshold = min_ssd * (1. + error)
    indices = np.where(ssd <= min_threshold)
    return indices

def findMatches(sample_image, window, window_size, pixel):
    print("Staring match finding")
    error_threshold = 0.1

    valid_mask = []
    columns = []
    for i in range(len(window)):
        columns = []
        for j in range(len(window)):
            if(window[i, j, :].any()):
                columns.append(1)
            else:
                columns.append(0)
        valid_mask.append(columns)

    gaussian_mask = getGaussian(window.shape[0], window_size / 6.4)
    weight = np.sum(gaussian_mask * valid_mask)

    

    sum_sqred_diffs = np.zeros((sample_image.shape[0], sample_image.shape[1]))
    sum_sqred_diff_arrays = []

    for ch in range(3):
        sum_sqred_diffs = np.zeros((sample_image.shape[0], sample_image.shape[1]))
        iterations = 0
        for i in range(sample_image.shape[0]):
            for j in range(sample_image.shape[1]):
                for ii in range(window.shape[0]):
                    for jj in range(window.shape[1]):
                        if(valid_mask[ii][jj] == 0 or i - ii < 0 or j - jj < 0):
                            continue
                        try:
                            distance = pow((window[ii, jj, ch] - sample_image[i - ii, j - jj, ch]), 2)
                        except:
                            #ca a l'aire stupide mais a mon avis ca marche
                            distance = pow((window[ii, jj, ch] - sample_image[i + ii, j + jj, ch]), 2)

                        sum_sqred_diffs[i, j] += distance * gaussian_mask[ii][jj]
                sum_sqred_diffs[i, j] /= weight
        sum_sqred_diff_arrays.append(sum_sqred_diffs)
    
    resultList = []

    sum = sum_sqred_diff_arrays[0] + sum_sqred_diff_arrays[1] + sum_sqred_diff_arrays[2]
    ta = sum[window_size // 2 + 1 : - window_size // 2 - 1]
    ta = [row[window_size // 2 + 1 : - window_size // 2 - 1] for row in ta]

    min = np.min(ta)

    for i in range(window_size // 2, sample_image.shape[0] - window_size // 2 + 1):
        for j in range(window_size // 2, sample_image.shape[1] - window_size // 2 + 1):
            if(sum[i, j] != 0):
                if(sum[i, j] <= min * (1 + error_threshold)):
                    resultList.append((i, j))
                    print((i, j))

    print("Stopped match finding")

    return (resultList)

def normalizedSsd(sample, window):
    wh, ww = window.shape[0], window.shape[1]
    sh, sw = sample.shape[0], sample.shape[1]

    mask = []
    columns = []
    for i in range(len(window)):
        columns = []
        for j in range(len(window)):
            if(window[i, j, :].any()):
                columns.append(1)
            else:
                columns.append(0)
        mask.append(columns)

    mask = np.array(mask)

    print(type(window))

    strided_sample = np.lib.stride_tricks.as_strided(sample, shape=((sh - wh + 1), (sw - ww + 1), wh, ww), 
                        strides=(sample.strides[0], sample.strides[1], sample.strides[0], sample.strides[1]))
    strided_sample = strided_sample.reshape(-1, wh, ww)

    strided_window = np.lib.stride_tricks.as_strided(window, shape=((sh - wh + 1), (sw - ww + 1), wh, ww),
                        strides=(0, 0, window.strides[0], window.strides[1]))
    strided_window = strided_window.reshape(-1, wh, ww)

    strided_mask = np.lib.stride_tricks.as_strided(mask, shape=((sh - wh + 1), (sw - ww + 1), wh, ww),
                        strides=(0, 0, mask.strides[0], mask.strides[1]))
    strided_mask = strided_mask.reshape(-1, wh, ww)

    sigma = wh / 6.4
    kernel = cv2.getGaussianKernel(ksize=wh, sigma=sigma)
    kernel_2d = kernel * kernel.T

    strided_kernel = np.lib.stride_tricks.as_strided(kernel_2d, shape=((sh - wh + 1), (sw - ww + 1), wh, ww),
                        strides=(0, 0, kernel_2d.strides[0], kernel_2d.strides[1]))
    strided_kernel = strided_kernel.reshape(-1, wh, ww)

    squared_differences = ((strided_sample - strided_window)**2) * strided_kernel * strided_mask
    ssd = np.sum(squared_differences, axis=(1, 2))
    ssd = ssd.reshape(sh - wh + 1, sw - ww + 1)

    total_ssd = np.sum(mask * kernel_2d)
    normalized_ssd = ssd / total_ssd

    return normalized_ssd

def getGaussian(window_size, sigma):
    x, y = np.meshgrid(np.arange(window_size), np.arange(window_size) + 1)
    x = x - window_size//2
    y = y - window_size//2
    return 1/(2*np.pi*sigma**2) * np.exp(-(x**2 + y**2)/(2*sigma**2))

def getNeighbourWindow(image, pixel, window_size):
    window = []
    for i in range(max(0, pixel[0] - window_size // 2), min(pixel[0] + window_size // 2, image.shape[0])):
        rows = []
        for j in range(max(0, pixel[1] - window_size // 2), min(pixel[1] + window_size // 2, image.shape[1])):
            colors = []
            for k in range(3):
                colors.append(image[i, j, k])
            rows.append(colors)
        window.append(rows)
    return window

# recherche des pixels qui sont voisins a l'extremité de l'image
def getUnfilledNeighbours(image):
    result = []
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if((image[i, j, 0] == image[i, j, 1] == image[i, j, 2] == 0)):
                try:
                    if(image[i-1 : i+2, j-1 : j+2, :].any()):
                        result.append((i, j))
                except IndexError:
                    print("pixel n'existe pas")
    return result

def myCountNonZero(image):
    result = 0
    for j in range(image.shape[0]):
        for k in range(image.shape[1]):
            if(not image[j, k, :].any()):
                continue
            else:
                result += 1
    return result // 3

window_size = 8
sample_image = cv2.imread(".\\128x128\\brick\\Brick_06-128x128.png") / 255
image_size = 50
image = np.zeros((sample_image.shape[0] + image_size, sample_image.shape[1] + image_size, 3))
sample_mid_size = 60 

image[image.shape[0] // 2 - sample_mid_size : image.shape[0] // 2 + sample_mid_size + 1, image.shape[1] // 2 - sample_mid_size : image.shape[1] // 2 + sample_mid_size + 1, :] = sample_image[sample_image.shape[0] // 2 - sample_mid_size : sample_image.shape[0] // 2 + sample_mid_size + 1, sample_image.shape[1] // 2 - sample_mid_size : sample_image.shape[1] // 2 + sample_mid_size + 1, :]

image = growImage(sample_image, image, window_size)

cv2.imwrite("sample.png", np.ciel(image * 255))
cv2.waitKey(0)
cv2.destroyAllWindows()

current pixel : 28x, 28 y
<class 'numpy.ndarray'>
current pixel : 28x, 29 y
<class 'numpy.ndarray'>
current pixel : 28x, 30 y
<class 'numpy.ndarray'>
current pixel : 28x, 31 y
<class 'numpy.ndarray'>
current pixel : 28x, 32 y
<class 'numpy.ndarray'>
current pixel : 28x, 33 y
<class 'numpy.ndarray'>
current pixel : 28x, 34 y
<class 'numpy.ndarray'>
current pixel : 28x, 35 y
<class 'numpy.ndarray'>
current pixel : 28x, 36 y
<class 'numpy.ndarray'>
current pixel : 28x, 37 y
<class 'numpy.ndarray'>
current pixel : 28x, 38 y
<class 'numpy.ndarray'>
current pixel : 28x, 39 y
<class 'numpy.ndarray'>
current pixel : 28x, 40 y
<class 'numpy.ndarray'>
current pixel : 28x, 41 y
<class 'numpy.ndarray'>
current pixel : 28x, 42 y
<class 'numpy.ndarray'>
current pixel : 28x, 43 y
<class 'numpy.ndarray'>
current pixel : 28x, 44 y
<class 'numpy.ndarray'>
current pixel : 28x, 45 y
<class 'numpy.ndarray'>
current pixel : 28x, 46 y
<class 'numpy.ndarray'>
current pixel : 28x, 47 y
<class 'numpy.ndarray'>


IndexError: index 178 is out of bounds for axis 1 with size 178

### Patch-based TS

In [ ]:
! ..\patch-based\texture_synthesis.py --image_path ../patch-based/data/input9.png --block_size 30 --num_block 14 --mode Best